In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Layer, Conv2D, BatchNormalization, GlobalAveragePooling2D,
                                     MaxPooling2D, Dense, InputLayer, add, Activation, Flatten)
import tensorflow.keras.regularizers as regularizers
import math
from functools import partial
(Ni)
input_shape = (224, 224, 3)
num_classes = 10

In [3]:
class ConvWithBatchNorm(Conv2D):
    """Conv layer with batch norm"""
    
    def __init__(self, activation='relu', name='convbn', **kwargs):
        """
        Initialize the layer.
        :param activation: Activation function (name or callable)
        :param name: Name suffix for the sub-layers
        :param kwargs: Mandatory and optional parameters of Conv2D
        """
        
        super().__init__(activation=None, name=name + '_c', **kwargs)
        self.activation = Activation(activation, name=name + '_act') \
                            if activation is not None else None
        self.batch_norm = BatchNormalization(name=name + '_b')     
        
    def call(self, inputs, training=None):
        """
        Call the layer operation.
        :param inputs: Input tensor to process
        :param training: Flag to let TF knows if it is a training iteration or not 
                         (this will affect the behavior of batch normalization)
        :return: Convolved tensor
        """
        
        x = super().call(inputs)
        x = self.batch_norm(x, training=training)
        x = self.activation(x) if self.activation is not None else x
        return x

In [4]:
class ResidualMerge(Layer):
    """Layer to merge the original tensor and the residual one in residual blocks"""
    
    def __init__(self, name, **kwargs):
        """
        Initializer the layer.
        :param name: Name suffix for the sub-layer
        :param kwargs: Optional parameters of Conv2D
        """
        
        super().__init__(name=name)
        self.shortcut = None
        self.kwargs = kwargs
        
    def build(self, input_shape):
        x_shape = input_shape[0]
        x_residual_shape = input_shape[1]
        if (x_shape[1] == x_residual_shape[1] and
            x_shape[2] == x_residual_shape[2] and
            x_shape[3] == x_residual_shape[3]):
            self.shortcut = partial(tf.identity, name=self.name + '_shortcut')
        else:
            strides = (int(round(x_shape[2] / x_residual_shape[2])),  # horizontal strides
                      int(round(x_shape[1] / x_residual_shape[1])))  # vertical strides
            num_channels = x_residual_shape[3]
            self.shortcut = ConvWithBatchNorm(filters=num_channels, kernel_size=(1, 1), 
                                              strides=strides, activation=None, **self.kwargs,
                                              name=self.name + '_shortcut_c')
    
    def call(self, inputs):
        """Call the layer.
        :param inputs: Tuples of two tensors to merge
        :return x_merge: Merged tensor
        """
        
        x, x_residual = inputs
        x_shortcut = self.shortcut(x)
        x_merge = add([x_shortcut, x_residual])
        
        return x_merge

In [5]:
class BasicResidualBlock(Model):
    """Basic residual block"""
    
    def __init__(self, filters=64, kernel_size=(3, 3), strides=1, activation='relu', 
                 kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4), 
                 name='res_basic', **kwargs):
        """Initialize the block
        :param filters: Number of filters
        :param kernel_size: Kernel size
        :param strides: Convolution strides
        :param activation: Activation function (name or callable)
        :param kernel_initializer: Kernel initialization method name
        :param kernel_regularizer: Kernel regularizer
        :param name: Name suffix for the sub-layers
        :param kwargs: Optional parameters of Conv2D
        """
        
        super().__init__(name=name)
        self.conv1 = ConvWithBatchNorm(filters=filters, kernel_size=kernel_size, strides=strides, 
                                      padding='same', activation=activation, name=name + '_cb_1',
                                      kernel_initializer=kernel_initializer, 
                                      kernel_regularizer=kernel_regularizer, **kwargs)
        
        self.conv2 = ConvWithBatchNorm(filters=filters, kernel_size=kernel_size, strides=1,
                                      padding='same', activation=None, name=name + '_cb_2',
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=kernel_regularizer, **kwargs)
        
        self.merge = ResidualMerge(name=name + '_merge', kernel_initializer=kernel_initializer,
                                  kernel_regularizer=kernel_regularizer)
        
        self.activation = Activation(activation, name=name + '_act')
        
    def call(self, inputs, training=None):
        """Call the layer
        :param inputs: Input tensor to process
        :param training: Flag to let TF knows if it is a training iteration or not
                         (this will affect the behavior of Batch Normalization)
        :return x_merge: Block output tensor
        """
        
        x = inputs
        
        # Residual path
        x_residual = self.conv1(x, training)
        x_residual = self.conv2(x_residual, training)
        
        # Merge residual result with original tensor
        x_merge = self.merge([x, x_residual])
        x_merge = self.activation(x_merge)
        
        return x_merge

In [6]:
class ResidualBlockWithBottleNeck(Model):
    """Residual block with bottleneck, recommended for deep ResNets (depth > 34)"""
    
    def __init__(self, filters=64, kernel_size=(3, 3), strides=1, activation='relu', 
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4),
                name='res_bottleneck', **kwargs):
        """Initializer the block
        :param filters: Number of filters
        :param kernel_size: Kernel size
        :param strides: Convolution strides
        :param activation: Activation function (name or callable)
        :param kernel_initializer: Kernel initialization method name
        :param kernel_regularizer: Kernel regularizer
        :param name: Name suffix for the sub-layers
        :param kwargs: Optional parameters of Conv2D
        """
        
        super().__init__(name=name)
        
        self.conv1 = ConvWithBatchNorm(filters=filters, kernel_size=(1, 1), strides=strides, 
                                      padding='valid', activation=activation, name=name + '_cb_1',
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=kernel_regularizer, **kwargs)
        
        self.conv2 = ConvWithBatchNorm(filters=filters, kernel_size=kernel_size, strides=1,
                                      padding='same', activation=activation, name=name + '_cb_2',
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=kernel_regularizer, **kwargs)
        
        self.conv3 = ConvWithBatchNorm(filters=filters * 4, kernel_size=(1, 1), strides=1,
                                      padding='valid', activation=None, name=name + '_cb_3',
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=kernel_regularizer, **kwargs)
        
        self.merge = ResidualMerge(name=name + '_merge', kernel_initializer=kernel_initializer,
                                  kernel_regularizer=kernel_regularizer)
        
        self.activation = Activation(activation, name=name + '_act')
        
    def call(self, inputs, training=None):
        """Call the layer
        :param inputs: Input tensor to process
        :param training: Flag to let TF knows if it is a training iteration or not
                         (this will affect the behavior of Batch Normalization)
        :return x_merge: Block ouput tensor"""
        
        x = inputs
        
        # Residual path
        x_residual = self.conv1(x, training)
        x_residual = self.conv2(x_residual, training)
        x_residual = self.conv3(x_residual, training)

        # Merge residual result with original tensor
        x_merge = self.merge([x, x_residual])
        x_merge = self.activation(x_merge)
        
        return x_merge

In [7]:
class MacroBlock(Sequential):
    """Macro-block, chaining multiple residual blocks (as a sequential model)"""
    
    def __init__(self, block_class=BasicResidualBlock, repetitions=2, filters=64, 
                kernel_size=(3, 3), strides=1, activation='relu', name='res_macroblock',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4),
                **kwargs):
        """Initialize the block
        :param block_class: Block class to be used
        :param repetitions: Number of times the block should be repeated inside
        :param filters: Number of filters
        :param kernel_size: Kernel size
        :param strides: Convolution strides
        :param activation: Activation function (name or callable)
        :param kernel_initializer: Kernel initialization method name
        :param kernel_regularizer: Kernel regularizer
        :param name: Name suffix for the sub-layers
        :param kwargs: Optional parameters of Conv2D"""
        
        model =[block_class(filters=filters, kernel_size=kernel_size, 
                            strides=strides if i == 0 else 1,
                            activation=activation, name='{}_{}'.format(name, i + 1),
                            kernel_initializer=kernel_initializer,
                            kernel_regularizer=kernel_regularizer, **kwargs) 
                for i in range(repetitions)]
        
        super().__init__(model, name=name)

In [8]:
class ResNet(Sequential):
    """ResNet model for classification"""
    
    def __init__(self, input_shape, num_classes=1000, block_class=BasicResidualBlock,
                 repetitions=(2, 2, 2, 2), kernel_initializer='he_normal', 
                 kernel_regularizer=regularizers.l2(1e-4), name='resnet'):
        """Initialize a ResNet model for classification
        :param input_shape: Input shape
        :param num_classes: Number of classes to predict
        :param block_class: Block class to be used
        :param repetitions: List of repetitions for each macro_blocks the network 
                            should contain
        :param kernel_initializer: Kernel initialization method name
        :param kernel_regularizer: Kernel regularizer
        :param name: Model's name"""
        
        filters, strides = 64, 2
        
        # Initial conv + max_pool layer
        conv_1 = [InputLayer(input_shape=input_shape),
                  ConvWithBatchNorm(filters=64, kernel_size=(7, 7), strides=strides, 
                                    padding='same', kernel_initializer=kernel_initializer,
                                    kernel_regularizer=kernel_regularizer, activation='relu',
                                    name='conv'),
                  MaxPooling2D(pool_size=3, strides=strides, padding='same', name='max_pool')]
        
        # Residual blocks
        res_blocks = [MacroBlock(block_class=block_class, repetitions=repet, 
                                 filters=min(filters * (2 ** i), 1024), kernel_size=(3, 3), 
                                 strides=1 if i == 0 else strides, activation='relu', 
                                 kernel_initializer=kernel_initializer, 
                                 kernel_regularizer=kernel_regularizer, name='block_{}'.format(i + 1)) 
                      for i, repet in enumerate(repetitions)]
        
        # Predict layer
        predict_layer = [GlobalAveragePooling2D(name='avg_pool'),
                        Dense(units=num_classes, activation='softmax', 
                             kernel_initializer=kernel_initializer, name=name + '_softmax')]
        
        super().__init__(conv_1 + res_blocks + predict_layer, name=name)

In [9]:
# Standard ResNet versions
class ResNet18(ResNet):
    
    def __init__(self, input_shape, num_classes=1000, name='resnet18',
                 kernel_initializer='he_normal', 
                 kernel_regularizer=regularizers.l2(1e-4)):
        super().__init__(input_shape=input_shape, num_classes=num_classes,
                        block_class=BasicResidualBlock, repetitions=(2, 2, 2, 2),
                        kernel_initializer=kernel_initializer,
                        kernel_regularizer=kernel_regularizer, name=name)
        
class ResNet34(ResNet):
    
    def __init__(self, input_shape, num_classes=1000, name='resnet34',
                kernel_initializer='he_normal', 
                kernel_regularizer=regularizers.l2(1e-4)):
        super().__init__(input_shape=input_shape, num_classes=num_classes,
                        block_class=BasicResidualBlock, repetitions=(3, 4, 6, 3),
                        kernel_initializer=kernel_initializer,
                        kernel_regularizer=kernel_regularizer, name=name)
        
class ResNet50(ResNet):
    
    def __init__(self, input_shape, num_classes=1000, name='resnet50',
                kernel_initializer='he_normal',
                kernel_regularizer=regularizers.l2(1e-4)):
        super().__init__(input_shape=input_shape, num_classes=num_classes,
                        block_class=ResidualBlockWithBottleNeck, 
                        repetitions=(3, 4, 6, 3), 
                        kernel_initializer=kernel_initializer,
                        kernel_regularizer=kernel_regularizer, name=name)
        
class ResNet101(ResNet):
    
    def __init__(self, input_shape, num_classes=1000, name='resnet101',
                kernel_initializer='he_normal',
                kernel_regularizer=regularizers.l2(1e-4)):
        super().__init__(input_shape=input_shape, num_classes=num_classes,
                        block_class=ResidualBlockWithBottleNeck, 
                        repetitions=(3, 4, 23, 3), 
                        kernel_initializer=kernel_initializer,
                        kernel_regularizer=kernel_regularizer, name=name)
        
class ResNet152(ResNet):
    
    def __init__(self, input_shape, num_classes=1000, name='resnet152',
                kernel_initializer='he_normal',
                kernel_regularizer=regularizers.l2(1e-4)):
        super().__init__(input_shape=input_shape, num_classes=num_classes,
                        block_class=ResidualBlockWithBottleNeck, 
                        repetitions=(3, 8, 36, 3), 
                        kernel_initializer=kernel_initializer,
                        kernel_regularizer=kernel_regularizer, name=name)

In [14]:
resnet50 = ResNet34(num_classes=100, input_shape=input_shape)
resnet50.summary(line_length=100)

Model: "resnet34"
____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
conv_c (ConvWithBatchNorm)                   (None, 112, 112, 64)                    9728           
____________________________________________________________________________________________________
max_pool (MaxPooling2D)                      (None, 56, 56, 64)                      0              
____________________________________________________________________________________________________
block_1 (MacroBlock)                         (None, 56, 56, 64)                      223104         
____________________________________________________________________________________________________
block_2 (MacroBlock)                         (None, 28, 28, 128)                     1119872        
_________________________________________________________________________